In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pprint import pprint
from glob import glob
import pkg_resources

from pymedext_eds.annotators import Endlines, SentenceTokenizer, Hypothesis, \
                                    ATCDFamille, SyntagmeTokenizer, Negation, RegexMatcher, \
                                    QuickUMLSAnnotator, Pipeline
from pymedext_eds.utils import rawtext_loader
from pymedext_eds.viz import display_annotations

In [3]:
data_path = pkg_resources.resource_filename('pymedext_eds', 'data/demo')
file_list = glob(data_path + '/*.txt')
chunk = [rawtext_loader(x) for x in file_list]

In [4]:
endlines = Endlines(['raw_text'], 'endlines', 'endlines:v1')
sentences = SentenceTokenizer(['endlines'], 'sentence', 'sentenceTokenizer:v1')
hypothesis = Hypothesis(['sentence'], 'hypothesis', 'hypothesis:v1')
family = ATCDFamille(['sentence'], 'context', 'ATCDfamily:v1')
syntagmes = SyntagmeTokenizer(['sentence'], 'syntagme', 'SyntagmeTokenizer:v1')
negation = Negation(['syntagme'], 'negation', 'Negation:v1')
regex = RegexMatcher(['endlines', 'syntagme'], 'regex', 'RegexMatcher:v1', 'list_regexp.json')


pipeline = Pipeline(pipeline = [endlines, sentences, hypothesis, family, syntagmes, negation, regex])

In [5]:
annotated_chunk = pipeline.annotate(chunk)

In [6]:
record = chunk[0].get_annotations('regex')[1].to_dict()

In [7]:
print(chunk[0].annotations[1].to_dict()['value'])

Antécédents médicaux 1,62 m pour 65,7 kg = IMC à 25,03.. En 1974 ,19 ans , , embolie pulmonaire compliquant une phlébite lors d'un alitement à la suite d’un accident de la voie publique. Des bilans d'hémostase effectués en 1995 et 2004 avaient éliminé une anomalie congénitale de la coagulation.. Insuffisance veineuse saphène interne bilatérale connue depuis 6 ans pour laquelle avait été proposée une intervention.. Mode de vie. Autonome et indépendante dans tous les gestes de la vie courante.. Loisirs : course à pied, semi-marathon, randonnées dans le cadre d'un club ainsi que pratique du vélo.. Le chirurgien conclue :. "Gonalgies droites évoluant depuis novembre 2012. Très sportive, a dû arrêter à cause de ses douleurs. Pas de blocage, ni d'instabilité. Escaliers difficiles. Réveil nocturne. A eu plusieurs infiltrations d’acide hyaluronique sans aucun effet.. Examen :. Bonnes amplitudes articulaires. Choc rotulien. Ligaments OK. Rotule OK. Douleurs palpation interne. Grinding test posi

In [8]:
print(chunk[0].annotations[0].to_dict()['value'])

Antécédents médicaux

1,62 m pour 65,7 kg = IMC à 25,03.
En 1974 (19 ans), embolie pulmonaire compliquant une phlébite lors d'un alitement à la suite d’un accident de la voie publique. Des bilans d'hémostase effectués en 1995 et 2004 avaient éliminé une anomalie congénitale de la coagulation.
Insuffisance veineuse saphène interne bilatérale connue depuis 6 ans pour laquelle avait été proposée une intervention.
Mode de vie

Autonome et indépendante dans tous les gestes de la vie courante.
Loisirs : course à pied, semi-marathon, randonnées dans le cadre d'un club ainsi que pratique du vélo.
Le chirurgien conclue :

"Gonalgies droites évoluant depuis novembre 2012. Très sportive, a dû arrêter à cause de ses douleurs. Pas de blocage, ni d'instabilité. Escaliers difficiles. Réveil nocturne. A eu plusieurs infiltrations d’acide hyaluronique sans aucun effet.

Examen :
Bonnes amplitudes articulaires. Choc rotulien. Ligaments OK. Rotule OK. Douleurs palpation interne. Grinding test positif (NB

In [151]:
def to_omop(record):
    s, e = record['span']
    
    attr = record['attributes']
    
    d = dict(
        lexical_variant=record['value'],
        start=s,
        end=e,
        offset=f"{s},{e}",
        snippet=attr['snippet'],
        term_modifiers=','.join([f'{k}={v}' for k, v in attr.items() if k not in {'snippet', 'label'}])
    )
    
    return d

In [152]:
to_omop(record)

{'lexical_variant': 'arythmie',
 'start': 3105,
 'end': 3113,
 'offset': '3105,3113',
 'snippet': "térielle à 60/40 mmHg, une fréquence cardiaque à 127/min en arythmie et une hypothermie à 35,5°C. L'auscultation pulmonaire retr",
 'term_modifiers': 'version=v2,id_regexp=id_regexp_troublerythme,hypothesis=certain,context=patient,negation=aff'}

In [153]:
import pandas as pd

In [154]:
pd.DataFrame.from_records([
    to_omop(ann.to_dict())
    for ann in chunk[1].get_annotations('regex')
]).sort_values('offset')

,lexical_variant,start,end,offset,snippet,term_modifiers
1,chirurgie,1340,1349,"1340,1349","faire une IRM"".. Le 5 février 2014, 2e consult...","version=v2,id_regexp=id_regexp_chir,hypothesis..."
2,chirurgie,1764,1773,"1764,1773","phie osseuse."". Le 19 février 2014, 3e consult...","version=v2,id_regexp=id_regexp_chir,hypothesis..."
3,chirurgie,2061,2070,"2061,2070","cortisonique."". Le 7 novembre 2014, 4e consult...","version=v2,id_regexp=id_regexp_chir,hypothesis..."
4,chirurgie,3041,3050,"3041,3050","ertise, interrogée sur le contenu de ses entre...","version=v2,id_regexp=id_regexp_chir,hypothesis..."
14,embolie pulmonaire,3138,3156,"3138,3156",affirme l’avoir informé de ses antécédents de ...,"version=v1,id_regexp=id_regexp_embolie,hypothe..."
5,chirurgie,3176,3185,"3176,3185",ents de phlébite et d’embolie pulmonaire.. Pou...,"version=v2,id_regexp=id_regexp_chir,hypothesis..."
15,Embolie pulmonaire,3654,3672,"3654,3672",ultation de préanesthésie. L’anesthésiste ment...,"version=v1,id_regexp=id_regexp_embolie,hypothe..."
16,embolie pulmonaire,3867,3885,"3867,3885",i par la patiente sont indiqués des antécédent...,"version=v1,id_regexp=id_regexp_embolie,hypothe..."
6,chirurgie,4000,4009,"4000,4009","e varice.. Le 19 janvier 2015, hospitalisation...","version=v2,id_regexp=id_regexp_chir,hypothesis..."
7,chirurgie,4561,4570,"4561,4570",'information dans laquelle il était indiqué de...,"version=v2,id_regexp=id_regexp_chir,hypothesis..."


In [145]:
pd.DataFrame.from_records([
    to_omop(ann.to_dict())
    for ann in chunk[1].get_annotations('regex')
]).sort_values('offset')

,lexical_variant,start,end,offset,snippet,term_modifiers
1,chirurgie,1334,1343,"1334,1343","le. Refaire une IRM"".. Le 5 février 2014, 2e c...","version=v2,id_regexp=id_regexp_chir,hypothesis..."
2,chirurgie,1750,1759,"1750,1759","xes. Scintigraphie osseuse."". Le 19 février 20...","version=v2,id_regexp=id_regexp_chir,hypothesis..."
3,chirurgie,2043,2052,"2043,2052","une infiltration cortisonique."". Le 7 novembr...","version=v2,id_regexp=id_regexp_chir,hypothesis..."
4,chirurgie,3013,3022,"3013,3022","é anesthésie.. Lors de l’expertise, interrogée...","version=v2,id_regexp=id_regexp_chir,hypothesis..."
14,embolie pulmonaire,3110,3128,"3110,3128","n orthopédiste, la patiente affirme l’avoir in...","version=v1,id_regexp=id_regexp_embolie,hypothe..."
5,chirurgie,3148,3157,"3148,3157",avoir informé de ses antécédents de phlébite e...,"version=v2,id_regexp=id_regexp_chir,hypothesis..."
15,Embolie pulmonaire,3626,3644,"3626,3644","x.. Le 12 janvier 2015, consultation de préane...","version=v1,id_regexp=id_regexp_embolie,hypothe..."
16,embolie pulmonaire,3838,3856,"3838,3856",e questionnaire médical rempli par la patiente...,"version=v1,id_regexp=id_regexp_embolie,hypothe..."
6,chirurgie,3971,3980,"3971,3980",des antécédents personnels de varice.. Le 19 ...,"version=v2,id_regexp=id_regexp_chir,hypothesis..."
7,chirurgie,4530,4539,"4530,4539","toires, était jointe une note d'information da...","version=v2,id_regexp=id_regexp_chir,hypothesis..."


In [155]:
display_annotations(chunk[0], ['regex'], attributes = ['negation', 'hypothesis', 'context'], label_key = 'label')